In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, WhiteKernel, ConstantKernel as C
from sklearn.preprocessing import StandardScaler, Normalizer
import ROOT
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.26/00


In [2]:
ROOT.gStyle.SetOptFit()

In [3]:
%jsroot on

In [4]:
ROOT.gInterpreter.ProcessLine('#include "../src/anaTSSA.h"')

0

In [5]:
from ROOT import pt_histo, plot_mass, pt_matrix, plot_matrix, plot_hist, integrate, unfold, asymmetry, plot_asym, pre_analysis, xf_histo, xf_matrix

# Analysis of TSSA

Backgroung of the data is predicted using Gaussian Process Regression(GPR) method with sidebands fitting. `scikit learn` package was used for this.
[More info](https://scikit-learn.org/stable/modules/gaussian_process.html).

In [6]:
def make_back(hist, hist_name, sideband, kernel, mass_min, mass_max, mass_bins):
    # print("GPR kernel : ", kernel)
    X = []
    for x in range(mass_bins):
        X.append([hist.GetBinCenter(x+1), hist.GetBinContent(x+1), hist.GetBinError(x+1)])
        
    data = np.array(X)
    
    
    mask = (((sideband[0] <= data[:, 0]) & (data[:, 0] <= sideband[1])) | ((sideband[2] <= data[:, 0]) & (data[:, 0] <= sideband[3])))

    X_train = data[mask]
    
    # define kernel
    if kernel == "RBF":
        tssa_kernel = C(500, (1e-10, 1e10))*RBF(1, (1e-20, 1e20))
        
    if kernel == "RationalQuadratic":
        tssa_kernel = C(1e5, (1e-7, 1e7))* RationalQuadratic(length_scale=1.0, alpha=0.1, alpha_bounds=(1e-5, 1e15))
        
    if kernel == "ExpSineSquared":
        tssa_kernel = C(1e5, (1e-7, 1e7))* ExpSineSquared(length_scale=1.0, periodicity=3.0, length_scale_bounds=(0.1, 10.0), periodicity_bounds=(10.0, 100.0))
        
    if kernel == "DotProduct":
        tssa_kernel = C(1e5, (1e-7, 1e7))* (DotProduct(sigma_0=1.0, sigma_0_bounds=(1.0, 100.0)) ** 2)
        
    if kernel == "Matern":
        tssa_kernel = C(500, (1e-5, 1e10))* Matern(length_scale=1e5, length_scale_bounds=(1e-20, 1e20), nu=2.5)
        
    # build the model
    tssa_model = GaussianProcessRegressor(kernel=tssa_kernel, alpha=X_train[:, 2]**2, n_restarts_optimizer=5000)
    
    # fit the model to predict the back ground
    tssa_model.fit(np.atleast_2d(X_train[:, 0]).T, X_train[:, 1]) 
    
    print(tssa_model.kernel_)
    
    print("score = ", tssa_model.score(np.atleast_2d(X_train[:, 0]).T, X_train[:, 1]))
    
    # y_pred, sigma = tssa_model.predict(np.atleast_2d(data[:, 0]).T, return_std=True)
    
    y_pred, cov_matrix = tssa_model.predict(np.atleast_2d(data[:, 0]).T, return_cov=True)
    
    sigma2 = np.sum(cov_matrix, axis=0)
    
    sigma = np.sqrt(abs(sigma2))

    
    # print(sigma)
    # print("********************")
    # print(np.sqrt(cov_matrix.diagonal()))
    
#     # print(cov_matrix)
#     plt.matshow(cov_matrix)
#     # plt.savefig("cov_matrix.png")
#     plt.show()
    
    hist1 = ROOT.TH1D(hist_name, "; mass(GeV/c^{2}); Yield", mass_bins, mass_min, mass_max)
    
    for x in range(mass_bins):
        hist1.SetBinContent(x+1, y_pred[x])
        hist1.SetBinError(x+1, sigma[x])
        
    return hist1

In [7]:
path = "../data_files/"

data = {}

for i in range(1, 6):
    for j in range(0, 2):
        data[i, 0, j] = path+"jpsi_"+str(i)+"_"+str(j)+".root"
        data[i, 1, j] = path+"drell_yan_"+str(i)+"_"+str(j)+".root"
        
data

{(1, 0, 0): '../data_files/jpsi_1_0.root',
 (1, 1, 0): '../data_files/drell_yan_1_0.root',
 (1, 0, 1): '../data_files/jpsi_1_1.root',
 (1, 1, 1): '../data_files/drell_yan_1_1.root',
 (2, 0, 0): '../data_files/jpsi_2_0.root',
 (2, 1, 0): '../data_files/drell_yan_2_0.root',
 (2, 0, 1): '../data_files/jpsi_2_1.root',
 (2, 1, 1): '../data_files/drell_yan_2_1.root',
 (3, 0, 0): '../data_files/jpsi_3_0.root',
 (3, 1, 0): '../data_files/drell_yan_3_0.root',
 (3, 0, 1): '../data_files/jpsi_3_1.root',
 (3, 1, 1): '../data_files/drell_yan_3_1.root',
 (4, 0, 0): '../data_files/jpsi_4_0.root',
 (4, 1, 0): '../data_files/drell_yan_4_0.root',
 (4, 0, 1): '../data_files/jpsi_4_1.root',
 (4, 1, 1): '../data_files/drell_yan_4_1.root',
 (5, 0, 0): '../data_files/jpsi_5_0.root',
 (5, 1, 0): '../data_files/drell_yan_5_0.root',
 (5, 0, 1): '../data_files/jpsi_5_1.root',
 (5, 1, 1): '../data_files/drell_yan_5_1.root'}

In [8]:
file = 1

ntype = 2
nspin = 2

pt_min = 0.0
pt_max = 5.0
pt_bins = 1

xf_min = 0.0
xf_max = 0.5
xf_bins = 1

pi = ROOT.TMath.Pi()
phi_bins = 15

mass_min = 1.0
mass_max = 6.0
mass_bins = 30

int_lumi = np.array([(3.92904e+08)/2, (9.69152e+08)/2])
exp_lumi = np.array([67704.8, 67704.8])
sideband = np.array([1.5, 2.5, 4.0, 5.0])

kernel = "RBF"

train_data = "train_data.root"

signal_iter = 3
back_iter = 3

## pT Analysis

In [9]:
matrix = {}
hist = {}
mass_hist = {}
back_hist = {}
signal_hist = {}
signal_phi = {}
back_phi = {}
unfo_signal = {}
unfo_back = {}
signal_asym = {}
back_asym = {}

In [10]:
pt_edge = np.linspace(pt_min, pt_max, pt_bins+1)
phi_edge = np.linspace(-pi, pi, phi_bins+1)
pt_width = (pt_max-pt_min)/pt_bins
phi_width = (2*pi)/phi_bins

In [11]:
for i in range(ntype):
    for j in range(nspin):
        for k in range(pt_bins):
            for x in range(phi_bins):
                hist_name = "hist"+str(i)+str(j)+str(k)+str(x)
                hist[i, j, k, x] = pt_histo(data[file, i, j], hist_name, pt_edge[k], pt_edge[k]+pt_width, phi_edge[x], phi_edge[x]+phi_width, mass_min, mass_max, mass_bins, int_lumi[i], exp_lumi[i])

In [12]:
for i in range(pt_bins):
    matrix_name = "matrix"+str(i)
    matrix[i] = pt_matrix(path+train_data, matrix_name, pt_edge[i], pt_edge[i]+pt_width, -pi, pi, phi_bins)

In [13]:
plot_matrix(matrix[0])

In [14]:
for i in range(nspin):
    for j in range(pt_bins):
        for k in range(phi_bins):
            mass_hist[i, j, k] = hist[(0, i, j, k)].Clone("mass_hist"+str(i)+str(j)+str(k))
            mass_hist[i, j, k].Add(hist[(1, i, j, k)])

In [15]:
for i in range(nspin):
    for j in range(pt_bins):
        for k in range(phi_bins):
            hist_name = "back_hist"+str(i)+str(j)+str(k)
            back_hist[i, j, k] = make_back(mass_hist[i, j, k], hist_name, sideband, kernel, mass_min, mass_max, mass_bins)

115**2 * RBF(length_scale=3.62)
score =  0.5942031924703747
135**2 * RBF(length_scale=4.07)
score =  0.6290711794688408
78.6**2 * RBF(length_scale=0.882)
score =  0.8326486181863152
122**2 * RBF(length_scale=5.11)
score =  0.7670163334799267
72.6**2 * RBF(length_scale=2.73)
score =  0.6020220419350579
116**2 * RBF(length_scale=4.51)
score =  0.833170904323129
102**2 * RBF(length_scale=4.88)
score =  0.45207522019193735
124**2 * RBF(length_scale=4.33)
score =  0.7098614357892992
159**2 * RBF(length_scale=2.77)
score =  0.520936142300039
219**2 * RBF(length_scale=3.68)
score =  0.9075005531440563
207**2 * RBF(length_scale=2.23)
score =  0.7303667988905693
275**2 * RBF(length_scale=3.24)
score =  0.7539744424232228
250**2 * RBF(length_scale=3.58)
score =  0.7872193674421746
182**2 * RBF(length_scale=3.57)
score =  0.8381422002633789
146**2 * RBF(length_scale=2.53)
score =  0.8239629465167014
158**2 * RBF(length_scale=3.28)
score =  0.7833810580906851
191**2 * RBF(length_scale=3.9)
score =

In [16]:
plot_mass(mass_hist[0, 0, 0], back_hist[0, 0, 0])

In [17]:
for i in range(nspin):
    for j in range(pt_bins):
        for k in range(phi_bins):
            hist_name = "signal_hist"+str(i)+str(j)+str(k)
            signal_hist[i, j, k] = mass_hist[i, j, k].Clone(hist_name)
            signal_hist[i, j, k].Add(back_hist[i, j, k], -1)

In [18]:
plot_hist(signal_hist[0, 0, 0])

In [19]:
for i in range(nspin):
    for j in range(pt_bins):
        hist_name = "signal_phi"+str(i)+str(j)
        signal_phi[i, j] = ROOT.TH1D(hist_name, "; #phi(rad); Yield", phi_bins, -pi, pi)
        for k in range(phi_bins):
            counts = integrate(signal_hist[i, j, k], sideband[1], sideband[2])
            signal_phi[i, j].SetBinContent(k+1, counts[0])
            signal_phi[i, j].SetBinError(k+1, counts[1])
            print("phi bin : ", k+1 ," jpsi count : ", counts[0])

phi bin :  1  jpsi count :  2072.6707637977397
phi bin :  2  jpsi count :  1462.5867797587603
phi bin :  3  jpsi count :  1160.843968789269
phi bin :  4  jpsi count :  457.1551435505756
phi bin :  5  jpsi count :  484.5335029529037
phi bin :  6  jpsi count :  959.3818422762771
phi bin :  7  jpsi count :  1414.448838868586
phi bin :  8  jpsi count :  1665.6674721217796
phi bin :  9  jpsi count :  2629.644816591538
phi bin :  10  jpsi count :  4882.41295139434
phi bin :  11  jpsi count :  10670.37758244412
phi bin :  12  jpsi count :  12589.21594086555
phi bin :  13  jpsi count :  9453.039635549725
phi bin :  14  jpsi count :  4329.548919437119
phi bin :  15  jpsi count :  2918.6762655412804
phi bin :  1  jpsi count :  2620.3087944320478
phi bin :  2  jpsi count :  4993.847653442698
phi bin :  3  jpsi count :  9180.575024669939
phi bin :  4  jpsi count :  11989.331286649469
phi bin :  5  jpsi count :  8866.672138306008
phi bin :  6  jpsi count :  3894.2455823765918
phi bin :  7  jpsi cou

In [20]:
for i in range(nspin):
    for j in range(pt_bins):
        hist_name = "back_phi"+str(i)+str(j)
        back_phi[i, j] = ROOT.TH1D(hist_name, "; #phi(rad); Yield", phi_bins, -pi, pi)
        for k in range(phi_bins):
            counts = integrate(back_hist[i, j, k], sideband[1], sideband[2])
            back_phi[i, j].SetBinContent(k+1, counts[0])
            back_phi[i, j].SetBinError(k+1, counts[1])

In [21]:
for i in range(nspin):
    for j in range(pt_bins):
        hist_name = "unfo_signal"+str(i)+str(j)
        unfo_signal[i, j] = unfold(matrix[j], signal_phi[i, j], hist_name, signal_iter)

Now unfolding...
Iteration : 0
Chi^2 of change 17086.9
Iteration : 1
Chi^2 of change 1212.17
Iteration : 2
Chi^2 of change 198.74
Calculating covariances due to number of measured events
Now unfolding...
Iteration : 0
Chi^2 of change 16687.2
Iteration : 1
Chi^2 of change 1158.18
Iteration : 2
Chi^2 of change 174.775
Calculating covariances due to number of measured events


In [22]:
for i in range(nspin):
    for j in range(pt_bins):
        hist_name = "unfo_back"+str(i)+str(j)
        unfo_back[i, j] = unfold(matrix[j], back_phi[i, j], hist_name, back_iter)

Now unfolding...
Iteration : 0
Chi^2 of change 1640.89
Iteration : 1
Chi^2 of change 84.2363
Iteration : 2
Chi^2 of change 8.76055
Calculating covariances due to number of measured events
Now unfolding...
Iteration : 0
Chi^2 of change 981.663
Iteration : 1
Chi^2 of change 48.2549
Iteration : 2
Chi^2 of change 4.19457
Calculating covariances due to number of measured events


In [23]:
func = ROOT.TF1("func", "[0]* sin(x)", -pi, pi)

In [24]:
for i in range(pt_bins):
    hist_name = "sigal_asym"+str(i)
    signal_asym[i] = asymmetry(unfo_signal[0, i], unfo_signal[1, i], hist_name)

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [25]:
for i in range(pt_bins):
    hist_name = "back_asym"+str(i)
    back_asym[i] = asymmetry(unfo_back[0, i], unfo_back[1, i], hist_name)

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [26]:
plot_asym(signal_asym[0], func)

 FCN=20.8894 FROM MIGRAD    STATUS=CONVERGED      14 CALLS          15 TOTAL
                     EDM=8.47236e-18    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.81463e-01   5.00191e-03   2.87510e-05   8.22965e-07


In [27]:
plot_asym(back_asym[0], func)

 FCN=6.44588 FROM MIGRAD    STATUS=CONVERGED      12 CALLS          13 TOTAL
                     EDM=8.31704e-16    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           5.40343e-01   1.75907e-02   2.34375e-05  -2.31855e-06


## xF Analysis

In [28]:
matrix1 = {}
hist1 = {}
mass_hist1 = {}
back_hist1 = {}
signal_hist1 = {}
signal_phi1 = {}
back_phi1 = {}
unfo_signal1 = {}
unfo_back1 = {}
signal_asym1 = {}
back_asym1 = {}

In [29]:
xf_edge = np.linspace(xf_min, xf_max, xf_bins+1)
xf_width = (xf_max-xf_min)/xf_bins

In [30]:
for i in range(ntype):
    for j in range(nspin):
        for k in range(xf_bins):
            for x in range(phi_bins):
                hist_name = "hist_xf"+str(i)+str(j)+str(k)+str(x)
                hist1[i, j, k, x] = xf_histo(data[file, i, j], hist_name, xf_edge[k], xf_edge[k]+xf_width, phi_edge[x], phi_edge[x]+phi_width, mass_min, mass_max, mass_bins, int_lumi[i], exp_lumi[i])

In [31]:
for i in range(xf_bins):
    matrix_name = "matrix_xt"+str(i)
    matrix1[i] = xf_matrix(path+train_data, matrix_name, xf_edge[i], xf_edge[i]+pt_width, -pi, pi, phi_bins)

In [32]:
plot_matrix(matrix1[0])

In [33]:
for i in range(nspin):
    for j in range(xf_bins):
        for k in range(phi_bins):
            mass_hist1[i, j, k] = hist1[(0, i, j, k)].Clone("mass_hist_xf"+str(i)+str(j)+str(k))
            mass_hist1[i, j, k].Add(hist1[(1, i, j, k)])

In [34]:
for i in range(nspin):
    for j in range(xf_bins):
        for k in range(phi_bins):
            hist_name = "back_hist_xf"+str(i)+str(j)+str(k)
            back_hist1[i, j, k] = make_back(mass_hist1[i, j, k], hist_name, sideband, kernel, mass_min, mass_max, mass_bins)

115**2 * RBF(length_scale=3.62)
score =  0.5941885523433352
135**2 * RBF(length_scale=4.07)
score =  0.6289134299557184
78.7**2 * RBF(length_scale=0.876)
score =  0.8338699939476958
122**2 * RBF(length_scale=5.1)
score =  0.7677670884938366
72.6**2 * RBF(length_scale=2.73)
score =  0.6020220463382615
116**2 * RBF(length_scale=4.5)
score =  0.8331628170179479
102**2 * RBF(length_scale=4.88)
score =  0.45207522097305586
124**2 * RBF(length_scale=4.33)
score =  0.7098614350728973
159**2 * RBF(length_scale=2.77)
score =  0.5209236317596704
219**2 * RBF(length_scale=3.68)
score =  0.9075093712345023
207**2 * RBF(length_scale=2.22)
score =  0.7307110961269936
276**2 * RBF(length_scale=3.25)
score =  0.7569887844042851
250**2 * RBF(length_scale=3.58)
score =  0.7871814502557533
182**2 * RBF(length_scale=3.57)
score =  0.8381422004788422
146**2 * RBF(length_scale=2.53)
score =  0.8239629455132274
158**2 * RBF(length_scale=3.28)
score =  0.7833810583795049
191**2 * RBF(length_scale=3.9)
score =

In [35]:
plot_mass(mass_hist1[0, 0, 0], back_hist1[0, 0, 0])

In [36]:
for i in range(nspin):
    for j in range(xf_bins):
        for k in range(phi_bins):
            hist_name = "signal_hist_xf"+str(i)+str(j)+str(k)
            signal_hist1[i, j, k] = mass_hist1[i, j, k].Clone(hist_name)
            signal_hist1[i, j, k].Add(back_hist1[i, j, k], -1)

In [37]:
plot_hist(signal_hist1[0, 0, 0])

In [38]:
for i in range(nspin):
    for j in range(xf_bins):
        hist_name = "signal_phi_xf"+str(i)+str(j)
        signal_phi1[i, j] = ROOT.TH1D(hist_name, "; #phi(rad); Yield", phi_bins, -pi, pi)
        for k in range(phi_bins):
            counts = integrate(signal_hist1[i, j, k], sideband[1], sideband[2])
            signal_phi1[i, j].SetBinContent(k+1, counts[0])
            signal_phi1[i, j].SetBinError(k+1, counts[1])

In [39]:
for i in range(nspin):
    for j in range(xf_bins):
        hist_name = "back_phi_xf"+str(i)+str(j)
        back_phi1[i, j] = ROOT.TH1D(hist_name, "; #phi(rad); Yield", phi_bins, -pi, pi)
        for k in range(phi_bins):
            counts = integrate(back_hist1[i, j, k], sideband[1], sideband[2])
            back_phi1[i, j].SetBinContent(k+1, counts[0])
            back_phi1[i, j].SetBinError(k+1, counts[1])

In [40]:
for i in range(nspin):
    for j in range(xf_bins):
        hist_name = "unfo_signal_xf"+str(i)+str(j)
        unfo_signal1[i, j] = unfold(matrix1[j], signal_phi1[i, j], hist_name, signal_iter)

Now unfolding...
Iteration : 0
Chi^2 of change 17201.9
Iteration : 1
Chi^2 of change 1226.12
Iteration : 2
Chi^2 of change 202.088
Calculating covariances due to number of measured events
Now unfolding...
Iteration : 0
Chi^2 of change 16815.4
Iteration : 1
Chi^2 of change 1174.08
Iteration : 2
Chi^2 of change 179.032
Calculating covariances due to number of measured events


In [41]:
for i in range(nspin):
    for j in range(xf_bins):
        hist_name = "unfo_back_xf"+str(i)+str(j)
        unfo_back1[i, j] = unfold(matrix1[j], back_phi1[i, j], hist_name, back_iter)

Now unfolding...
Iteration : 0
Chi^2 of change 1639.96
Iteration : 1
Chi^2 of change 84.4018
Iteration : 2
Chi^2 of change 8.75435
Calculating covariances due to number of measured events
Now unfolding...
Iteration : 0
Chi^2 of change 982.086
Iteration : 1
Chi^2 of change 48.3852
Iteration : 2
Chi^2 of change 4.19851
Calculating covariances due to number of measured events


In [42]:
for i in range(xf_bins):
    hist_name = "sigal_asym_xf"+str(i)
    signal_asym1[i] = asymmetry(unfo_signal1[0, i], unfo_signal1[1, i], hist_name)

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [43]:
for i in range(xf_bins):
    hist_name = "back_asym_xf"+str(i)
    back_asym1[i] = asymmetry(unfo_back1[0, i], unfo_back1[1, i], hist_name)

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


In [44]:
plot_asym(signal_asym1[0], func)

 FCN=21.104 FROM MIGRAD    STATUS=CONVERGED      14 CALLS          15 TOTAL
                     EDM=2.1537e-14    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.81560e-01   4.98095e-03   1.14345e-05  -4.16673e-05


In [45]:
plot_asym(back_asym1[0], func)

 FCN=6.43859 FROM MIGRAD    STATUS=CONVERGED      12 CALLS          13 TOTAL
                     EDM=1.909e-15    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           5.40206e-01   1.75823e-02   2.34149e-05   3.51433e-06
